In [26]:
import os
import sys

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score,f1_score
from sklearn.metrics import classification_report, roc_curve, average_precision_score, precision_recall_curve



https://www.kaggle.com/code/qqzzxxdd/private-score-0-59168-simple-fe-autogluon

In [2]:
train = pd.read_csv('train.csv/train.csv')
test = pd.read_csv('test.csv/test.csv')

print("shape of the train", train.shape)
print("shape of the test", test.shape)

shape of the train (26570, 26)
shape of the test (20775, 25)


In [3]:
print("number of unique rows in train",len(train["id"].unique()))
print("number of unique rows in test", len(test["id"].unique()))

number of unique rows in train 26570
number of unique rows in test 20775


In [4]:
train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [5]:
#Take the train df without target [failure]
train_df = train.iloc[0:,0:-1]

In [10]:
# now concate the train and test df so that we can can make single dataset (train+test)
df_full = pd.concat([train, test])
df_full.shape

(47345, 26)

In [16]:
df_full.isnull().sum()

id                    0
product_code          0
loading             473
attribute_0           0
attribute_1           0
attribute_2           0
attribute_3           0
measurement_0         0
measurement_1         0
measurement_2         0
measurement_3       710
measurement_4       947
measurement_5      1184
measurement_6      1420
measurement_7      1657
measurement_8      1894
measurement_9      2131
measurement_10     2367
measurement_11     2604
measurement_12     2841
measurement_13     3077
measurement_14     3314
measurement_15     3551
measurement_16     3788
measurement_17     4024
failure           20775
m_3_missing           0
dtype: int64

In [14]:
# making m_3_missing feature from null 
df_full["m_3_missing"] = df_full.measurement_3.isna()
df_full["m_3_missing"] = df_full.measurement_5.isna()

In [13]:
df_full.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,m_3_missing
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0.0,False
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0.0,False
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0.0,False
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0.0,False
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0.0,False


In [21]:
null_feature = [col for col in df_full.columns if df_full[col].isnull().sum() != 0]
not_null_featur = [col for col in df_full.columns if col not in null_feature]
object_feature = [col for col in train.columns if train[col].dtype=="object"]

In [20]:
not_null_featur

['id',
 'product_code',
 'attribute_0',
 'attribute_1',
 'attribute_2',
 'attribute_3',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'm_3_missing']

In [35]:
#!git clone https://github.com/analokmaus/kuma_utils.git
!git clone https://github.com/analokmaus/kuma_utils.git
#!pip install utils.coco_utils
#!pip install imutils
#!pip install lightgbm 

'git' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
sys.path.append("kuma_utils/")

In [47]:
from kuma_utils.preprocessing.imputer import LGBMImputer


ModuleNotFoundError: No module named 'kuma_utils'

In [48]:
display(df_full['product_code'].unique())
df_A = df_full[df_full['product_code']=="A"]
df_B = df_full[df_full['product_code']=="B"]
df_C = df_full[df_full['product_code']=="C"]
df_D = df_full[df_full['product_code']=="D"]
df_E = df_full[df_full['product_code']=="E"]
df_F = df_full[df_full['product_code']=="F"]
df_G = df_full[df_full['product_code']=="G"]
df_H = df_full[df_full['product_code']=="H"]
df_I = df_full[df_full['product_code']=="I"]

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype=object)

In [49]:
# all_features_ = pd.concat([train_iterimp_A,train_iterimp_B,train_iterimp_C,train_iterimp_D,
#                            train_iterimp_E,test_iterimp_F,test_iterimp_G,test_iterimp_H,test_iterimp_I
#                           ])
# all_features = pd.concat([all_features_,all_features[nonull_features]],axis = 1)

In [50]:
display(df_full['product_code'].unique())

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype=object)

In [52]:
df_full.isnull().sum()

id                    0
product_code          0
loading             473
attribute_0           0
attribute_1           0
attribute_2           0
attribute_3           0
measurement_0         0
measurement_1         0
measurement_2         0
measurement_3       710
measurement_4       947
measurement_5      1184
measurement_6      1420
measurement_7      1657
measurement_8      1894
measurement_9      2131
measurement_10     2367
measurement_11     2604
measurement_12     2841
measurement_13     3077
measurement_14     3314
measurement_15     3551
measurement_16     3788
measurement_17     4024
failure           20775
m_3_missing           0
dtype: int64